In [1]:
import re
import pprint
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from math import sqrt
import warnings
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pyproj as proj
import copy
import os
import folium
from IPython import display

### This code is for the other models, it was done going into the tail end of the project, so there are definitely perfections that can be made here, I would like to spend some time cleaning this up in the near future.

In [2]:
desired = [
 'property_type','room_type','accommodates','bathrooms',
 'bedrooms','beds','price','cleaning_fee', 'latitude', 'longitude', 'amenities']

def clean_price(strngy):
    """
    Quick function to strip formatting from dollars
    """
    dols = strngy.replace('$', '')
    dols = dols.replace(',', '')
    dols = float(dols)
    return dols

def clean_amens(amenity):
    """
    Quick function to unpack amenities into list
    """
    amenity = amenity.replace('{','')
    amenity = amenity.replace('}','')
    amenity = amenity.replace('"','')
    amenlist = amenity.split(',')
    return amenlist
    
def clean_DF(DF, region):
    """
    Clean_DF does what it says, it takes in a cityDF, then initially drops any columns
    that won't be used, and proceeds to fill out ones that will be needed with some 
    assumed values.
    """
    #Dropping undesirable columns...
    DF = DF[desired]
    
    #Fill NLPCols with dummy Polish text
    #DF.name.fillna(value = 'BezNazwy', inplace = True)
    #DF.summary.fillna(value = 'BezPodsumowania', inplace = True)
    #DF.space.fillna(value = 'BezCosmosu', inplace = True)
    #DF.description.fillna(value = 'BezInformacji', inplace = True)
    #DF.neighborhood_overview.fillna(value = 'BezSasiada', inplace = True)
    #DF.notes.fillna(value = 'BezNotatki', inplace = True)
    #DF.transit.fillna(value = 'BezDrog', inplace = True)
    
    #Fill DateCols with dummy date + convert to dt object
    #DF.host_since.fillna(value = '2018-05-26', inplace = True)
    #DF.first_review.fillna(value = '2018-5-26', inplace = True)
    #DF.last_review.fillna(value = '2018-5-26', inplace = True)

    #DF.host_since = pd.to_datetime(DF.host_since)
    #DF.first_review = pd.to_datetime(DF.first_review)
    #DF.last_review = pd.to_datetime(DF.last_review)

    #DF['host_days'] = (DF['last_review'] - DF['host_since']).apply(lambda x: float(x.days))
    #DF['list_days'] = (DF['last_review'] - DF['first_review']).apply(lambda x: float(x.days))
    #DF = DF.drop(labels = ['host_since', 'first_review', 'last_review'], axis=1)
    
    #Fill Booleans (assumption that if no t/f it is false)
    #DF.host_is_superhost.fillna(value = 'f', inplace = True)

    #DF.host_is_superhost = np.where(DF.host_is_superhost == 't', 1, 0)
    #DF.instant_bookable = np.where(DF.instant_bookable == 't', 1, 0)
    #DF.require_guest_profile_picture = np.where(DF.require_guest_profile_picture == 't', 1, 0)
    #DF.require_guest_phone_verification = np.where(DF.require_guest_phone_verification == 't', 1, 0)
      
    #Fill Price-Related numbers with assumption that if a weekly price is not offered
    #that I would just fill that in with the number of 
    #DF.weekly_price.fillna(value = '$0.00', inplace = True)
    #DF.monthly_price.fillna(value = '$0.00', inplace = True)
    #DF.security_deposit.fillna(value = '$0.00', inplace = True)
    DF.cleaning_fee.fillna(value = '$0.00', inplace = True)

    DF.price = DF.price.apply(clean_price)
    #DF.weekly_price = DF.weekly_price.apply(clean_price)
    #DF.monthly_price = DF.monthly_price.apply(clean_price)
    #DF.security_deposit = DF.security_deposit.apply(clean_price)
    DF.cleaning_fee = DF.cleaning_fee.apply(clean_price)
    #DF.extra_people = DF.extra_people.apply(clean_price)

    #DF.weekly_price = np.where(DF.weekly_price == 0, DF.price * 7, DF.weekly_price)
    #DF.monthly_price = np.where(DF.monthly_price == 0, DF.price * 30, DF.monthly_price)
    DF = DF[DF['price'] != 0]
    
    #Unpack Amenities list 
    DF['amenlist'] = DF['amenities'].map(clean_amens)
    DF['amenlen'] = DF['amenlist'].map(len)
    
    #Make sure to only use spots in the data where bathrooms, bedrooms, and beds included
    DF = DF[np.isfinite(DF.bathrooms)]
    DF = DF[np.isfinite(DF.bedrooms)]
    DF = DF[np.isfinite(DF.beds)]
    
    #Drop Non Prop Types
    prop_types = ['Apartment', 'House', 'Townhouse', 'Loft', 'Condominium']
    DF = DF[DF['property_type'].isin(prop_types)]
    
    #Append region label to make final DF less confusing
    #DF['region'] = region
    
    return DF

In [3]:
Austin = pd.read_csv('data/AustinListings.csv')
Boston = pd.read_csv('data/BostonListings.csv')
DC = pd.read_csv('data/DCListings.csv')
LA = pd.read_csv('data/LAListings.csv')
NewOrleans = pd.read_csv('data/NewOrleansListings.csv')
NY = pd.read_csv('data/NYListings.csv')
Portland = pd.read_csv('data/PortlandListings.csv')
Seattle = pd.read_csv('data/SeattleListings.csv')
SF = pd.read_csv('data/SFListings.csv')
dirtylst = [Austin, Boston, DC, LA, NewOrleans, NY, Portland, Seattle, SF]

aus = clean_DF(Austin, 'Austin')
bos = clean_DF(Boston, 'Boston')
wdc = clean_DF(DC, 'DC')
los = clean_DF(LA, 'Los Angeles')
nla = clean_DF(NewOrleans, 'New Orleans')
nyc = clean_DF(NY, 'New York City')
prt = clean_DF(Portland, 'Portland')
sea = clean_DF(Seattle, 'Seattle')
sfo = clean_DF(SF, 'San Francisco')
nlst = ['Austin', 'Boston', 'Washington DC', 'Los Angeles', 'New Orleans', 'New York City', 'Portland', 'Seattle', 'San Francisco']
dfl = [nyc, aus, bos, wdc, los, nla, prt, sea, sfo]

In [4]:
for item in dfl:
    item['adjlat'] = pd.cut(item.latitude, 200, labels = range(0,200))
    item['adjlon'] = pd.cut(item.longitude, 200, labels = range(0,200))

In [5]:
#If no good, adjlat = normlat

for item in dfl:
    adjlat = np.array(item.latitude)/np.mean(item.latitude)
    adjlon = np.array(item.longitude)/np.mean(item.longitude)
    
    item['adjlat'] = np.round(adjlat, 4) 
    item['adjlon'] = np.round(adjlon, 4)

#### The main idea of the code below is that I unpack the amenities column of the tables into individual amenities to see if they have any solid feature importance, then I show the top 20 features, and finally I truncate down to only the ten most important features to try to get a more precise model. This code trains over each city individually, the count is merely there to print the corresponding selection from nlst when the scores and feature importances are being printed.

In [7]:
count = 0

for item in dfl:
    rfamens = pd.get_dummies((item.amenlist).apply(pd.Series).stack()).sum(level = 0)
    
    rfamenity = pd.concat([rfamens, item.price, item.cleaning_fee, item.bathrooms, item.bedrooms, item.beds, item.adjlat, item.adjlon, item.amenlen], axis = 1)
    rfamenity['fullprice'] = rfamenity.price + rfamenity.cleaning_fee

    
    rfamenity = rfamenity[rfamenity['price'] <= 500]
    item = item[item['bedrooms'] <= 5]
    item = item[item['beds'] <= 6]
    item = item[item['bathrooms'] <= 4]
    item = item[item['accommodates'] <= 10]
    
    adjlat = np.array(item.latitude)/np.mean(item.latitude)
    adjlon = np.array(item.longitude)/np.mean(item.longitude)
    
    item['adjlat'] = np.round(adjlat, 4) 
    item['adjlon'] = np.round(adjlon, 4)
    
    rfamenity['pricecut'] = pd.cut(rfamenity.price, bins = [1, 150, 300, 400, 500], right = True)
    rfamenity['pricecut'] = rfamenity['pricecut'].map(str)
    rfamenity['adjlat'] = rfamenity['adjlat']
    rfamenity['adjlon'] = rfamenity['adjlon']

    

    xamens = rfamenity.drop(labels = ['pricecut', 'cleaning_fee', 'fullprice', 'price', 'translation missing: en.hosting_amenity_50', 'translation missing: en.hosting_amenity_49'], axis = 1, errors = 'ignore')
    yamens = rfamenity['pricecut']

    x_train, x_test, y_train, y_test = train_test_split(xamens, yamens, test_size = .4)

    rf = RandomForestClassifier(n_estimators=1000, criterion ='entropy', min_samples_split=2,
                        min_samples_leaf=50, n_jobs=16, bootstrap=True, random_state=42)

    rf.fit(x_train, y_train)
    rf.score(x_test, y_test)
    ypred = rf.predict(x_test)
    print(rf.score(x_test, y_test))
    print(f1_score(y_test, ypred, average = 'macro'))
    print(f1_score(y_test, ypred, average = 'weighted'))
    
    feature_importances = pd.DataFrame(rf.feature_importances_,
                                  index = xamens.columns,
                                  columns=['importance']).sort_values('importance', ascending=False)
    feats = feature_importances.head(20)
    print(feats)
    
    print('\n\n\n This is the end of ' + nlst[count] + ' all features. \n\n\n')

    tops = feature_importances.index.tolist()[:10]
    
    
    rftop = rfamenity[tops]
    xamens = rftop
    yamens = rfamenity['pricecut']
    
    x_train, x_test, y_train, y_test = train_test_split(xamens, yamens, test_size = .4)

    rf = RandomForestClassifier(n_estimators=1500, criterion ='entropy', min_samples_split=2,
                        min_samples_leaf=50, n_jobs=16)

    rf.fit(x_train, y_train)
    rf.score(x_test, y_test)
    ypred = rf.predict(x_test)
    print(rf.score(x_test, y_test))
    print(f1_score(y_test, ypred, average = 'macro'))
    print(f1_score(y_test, ypred, average = 'weighted'))
    
    feature_importances = pd.DataFrame(rf.feature_importances_,
                                  index = xamens.columns,
                                  columns=['importance']).sort_values('importance', ascending=False)
    feats = feature_importances.head(20)
    print(feats)
    print('\n\n\n This is the end of ' + nlst[count] + ' top features. \n\n\n')
    count+=1

0.7677440137442285
0.30406364712324063
0.7140445102379229
                          importance
bedrooms                    0.186999
adjlon                      0.180950
beds                        0.150126
adjlat                      0.094341
Family/kid friendly         0.053468
TV                          0.047842
bathrooms                   0.037681
Cable TV                    0.026947
Lock on bedroom door        0.023919
amenlen                     0.019256
Elevator                    0.017057
Air conditioning            0.016777
Dryer                       0.016579
Washer                      0.015015
Doorman                     0.011620
Buzzer/wireless intercom    0.005909
Gym                         0.005553
Hair dryer                  0.005420
Internet                    0.005381
Dishwasher                  0.005209



 This is the end of Austin all features. 



0.7946955868141308
0.364068162646185
0.7678039180260708
                      importance
adjlon                  0.26

0.8248309772587584
0.2410811200570664
0.7503173147345884
                            importance
bedrooms                      0.216922
beds                          0.176710
bathrooms                     0.133784
Family/kid friendly           0.045696
Elevator                      0.043388
adjlon                        0.034105
TV                            0.029244
adjlat                        0.025088
Cable TV                      0.022507
Air conditioning              0.021798
amenlen                       0.017760
Pets live on this property    0.015900
Kitchen                       0.015507
Dryer                         0.013686
Washer                        0.013098
First aid kit                 0.011181
Free parking on premises      0.010616
Indoor fireplace              0.009876
Gym                           0.009222
Lock on bedroom door          0.007842



 This is the end of Portland all features. 



0.8494161032575291
0.3376953818827708
0.8211863633336645
                 